<left>FINM 36702 - Portfolio Credit Risk: Modeling and Estimation</left>
<left>Spring 2023</left>
<br>
<h1><center> Assignment 1 </center></h1>
<center>Due - 18:00 [CST] March 30th, 2023</center>
<br>
<h3>Ki Hyun</h3>
<h3>Student ID: 12125881</h3>

### Imports

In [1]:
from scipy.stats import multivariate_normal
from scipy.optimize import minimize
from scipy.stats import norm
from scipy.stats import mvn
import numpy as np
import warnings

In [2]:
warnings.filterwarnings("ignore")

### Helper Functions

In [3]:
def MVN_p(rho, pd_1, pd_2):
    try:
        rho = rho[0]
    except:
        pass
    mu = np.array([0, 0], dtype = float)
    sigma = np.array([[1, rho], [rho, 1]], dtype = float)
    inv_cdf_1 = norm.ppf(pd_1)
    inv_cdf_2 = norm.ppf(pd_2)

    # add a small positive value to the diagonal entries of the covariance matrix
    sigma += np.eye(2) * 1e-8

    cdf = multivariate_normal(mean = mu, cov = sigma).cdf([inv_cdf_1, inv_cdf_2])

    return cdf

In [4]:
def f(x, pd_1, pd_2, pdj):
    return abs(MVN_p(x, pd_1, pd_2) - pdj)

In [5]:
def MVN_bounded_p(rho, low_1, upper_1, low_2, upper_2):
    mu = np.asarray([0, 0], dtype = float)
    cov = np.asarray([[1, rho], [rho, 1]], dtype = float)

    # Define the lower and upper bounds for the two variables
    lower = np.array([norm.ppf(low_1), norm.ppf(low_2)], dtype = float)
    upper = np.array([norm.ppf(upper_1), norm.ppf(upper_2)], dtype = float)

    # Calculate the joint probability using the mvn function
    joint_prob, _ = mvn.mvnun(lower, upper, mu, cov)
    
    return joint_prob

## Question 1.

In [6]:
PDs = [0.1, 0.2, 0.3]

In [7]:
PDJ = 0.06

In [8]:
rhos = []

In [9]:
for i in range(len(PDs) - 1):
    for j in range(i + 1, len(PDs)):
        rho = minimize(f, x0 = 0, args = (PDs[i], PDs[j], PDJ), bounds = [(-1, 1)])
        rhos.append(rho)
        print("The correlation betweeen firm", i + 1, "and firm", j + 1, "is:", rho.x[0])
        Corr_D = (PDJ - PDs[i] * PDs[j]) / ((PDs[i] * (1 - PDs[i]) * PDs[j] * (1 - PDs[j])) ** 0.5)
        print("The default correlation betweeen firm", i + 1, "and firm", j + 1, "is:", Corr_D)

The correlation betweeen firm 1 and firm 2 is: 0.6025696613871128
The default correlation betweeen firm 1 and firm 2 is: 0.33333333333333326
The correlation betweeen firm 1 and firm 3 is: 0.43224726736836194
The default correlation betweeen firm 1 and firm 3 is: 0.2182178902359924
The correlation betweeen firm 2 and firm 3 is: -6.532844441960453e-09
The default correlation betweeen firm 2 and firm 3 is: 0.0


## Question 2.

In [10]:
rho_1_2 = 0.4
rho_1_3 = 0.5
rho_2_3 = 0.6
PD = 0.1

In [11]:
mu = [0,0,0]
sigma = [[1,0.4,0.5],[0.4,1,0.6],[0.5,0.6,1]]

In [12]:
print("The PDJ between firm 1 and firm 2 is", round(MVN_p(rho_1_2, PD, PD), 3))
print("The PDJ between firm 1 and firm 3 is", round(MVN_p(rho_1_3, PD, PD), 3))
print("The PDJ between firm 2 and firm 3 is", round(MVN_p(rho_2_3, PD, PD), 3))

The PDJ between firm 1 and firm 2 is 0.027
The PDJ between firm 1 and firm 3 is 0.032
The PDJ between firm 2 and firm 3 is 0.039


In [13]:
print("The probability that all three default under the Gauss copula", 
      round(multivariate_normal(mean = mu, cov = sigma).cdf([norm.ppf(PD), norm.ppf(PD), norm.ppf(PD)]),3))

The probability that all three default under the Gauss copula 0.016


## Question 3.

In [14]:
transition_matrix = [[[0.5, 1], [0.1, 0.5], [0, 0.1]], [[0.7, 1], [0.2, 0.7], [0, 0.2]]]

In [15]:
ratings = ["A", "B", "D"]

In [16]:
rho = 0.4

In [17]:
for i in range(len(transition_matrix[0])):
    for j in range(len(transition_matrix[0])):
        print("Joint Probability of firm", ratings[0], "-> rating", ratings[i], 
              "and firm", ratings[1], "-> rating", ratings[j], ":", 
              round(MVN_bounded_p(rho, transition_matrix[0][i][0], transition_matrix[0][i][1],
                           transition_matrix[0][j][0], transition_matrix[0][j][1]), 2))

Joint Probability of firm A -> rating A and firm B -> rating A : 0.32
Joint Probability of firm A -> rating A and firm B -> rating B : 0.16
Joint Probability of firm A -> rating A and firm B -> rating D : 0.02
Joint Probability of firm A -> rating B and firm B -> rating A : 0.16
Joint Probability of firm A -> rating B and firm B -> rating B : 0.19
Joint Probability of firm A -> rating B and firm B -> rating D : 0.05
Joint Probability of firm A -> rating D and firm B -> rating A : 0.02
Joint Probability of firm A -> rating D and firm B -> rating B : 0.05
Joint Probability of firm A -> rating D and firm B -> rating D : 0.03


## Question 4.

In [18]:
PDs2 = [0.01, 0.02, 0.03, 0.04]

In [19]:
PDJ2 = 0.001

In [20]:
rhos2 = []

In [21]:
for i in range(len(PDs2) - 1):
    for j in range(i + 1, len(PDs2)):
        rho = minimize(f, x0 = 0, args = (PDs2[i], PDs2[j], PDJ2), method='L-BFGS-B', bounds = [(-1, 1)])
        rhos2.append(rho)
        print("The correlation betweeen firm", i + 1, "and firm", j + 1, "is:", rho.x[0])

The correlation betweeen firm 1 and firm 2 is: 0.3111861238586191
The correlation betweeen firm 1 and firm 3 is: 0.23718940120836163
The correlation betweeen firm 1 and firm 4 is: 0.18336733505639266
The correlation betweeen firm 2 and firm 3 is: 0.1005098252975813
The correlation betweeen firm 2 and firm 4 is: 0.04430852169794148
The correlation betweeen firm 3 and firm 4 is: -0.03622909563652467


In [22]:
Sigma = []

In [23]:
for i in range(len(PDs2)):
    Sigma_row = []
    for j in range(len(PDs2)):
        if i == j:
            Sigma_row.append(1.0)

        else:
            low = min(i, j) + 1
            high = max(i, j) + 1
            n = len(PDs)
            if low == 1:
                flag = int(high - low - 1)
            else:
                flag = int(n * (low - 1) - low * (low - 1) / 2 + (high - low - 1))
            Sigma_row.append(rhos2[flag].x[0])

    Sigma.append(Sigma_row)

In [24]:
Sigma = np.array(Sigma, dtype = float)

In [25]:
w, v = np.linalg.eig(Sigma)

In [26]:
print('Eigen-values:', w)

Eigen-values: [1.5796624  0.66913805 0.82730131 0.92389824]


In [27]:
print('Eigen-vectors:', v)

Eigen-vectors: [[-0.59654517 -0.77638017  0.19575699 -0.0551987 ]
 [-0.53394075  0.56906858  0.54175004 -0.31237017]
 [-0.47608928  0.17956563 -0.81600203 -0.27429157]
 [-0.36384081  0.20285702 -0.04823672  0.90782273]]
